#   consomotion Simple

In [ ]:
import os
import base64
import requests
import json
from google.transit import gtfs_realtime_pb2  
from google.protobuf.json_format import MessageToJson

os.environ["CLIENT_ID"] = "6722072041100899" 
os.environ["CLIENT_SECRET"] = "ISo0oqPMkfCB8wvaPSxrfj20EmnuvKwE"  


client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

if not client_id or not client_secret:
    print("Error: CLIENT_ID and CLIENT_SECRET must be set as environment variables.")
    exit()


auth_string = f"{client_id}:{client_secret}"
encoded_auth = base64.b64encode(auth_string.encode()).decode()


city = "tampere"  
endpoint = "vehicleposition"  
url = f"https://data.waltti.fi/{city}/api/gtfsrealtime/v1.0/feed/{endpoint}"

headers = {
    "Authorization": f"Basic {encoded_auth}",
}


try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()  


    feed = gtfs_realtime_pb2.FeedMessage()
    feed.ParseFromString(response.content)
    feed_json = json.loads(MessageToJson(feed))

    print(json.dumps(feed_json, indent=4))

except requests.exceptions.RequestException as e:
    print(json.dumps({"error": f"Error connecting to the API: {e}"}, indent=4))
except Exception as e:
    print(json.dumps({"error": f"Unexpected error: {e}"}, indent=4))


{
    "header": {
        "gtfsRealtimeVersion": "2.0",
        "incrementality": "FULL_DATASET",
        "timestamp": "1732310039"
    },
    "entity": [
        {
            "id": "47374_43",
            "vehicle": {
                "trip": {
                    "tripId": "73_14674_8544710",
                    "startTime": "23:10:00",
                    "startDate": "20241122",
                    "scheduleRelationship": "SCHEDULED",
                    "routeId": "5047374",
                    "directionId": 0
                },
                "position": {
                    "latitude": 61.32567,
                    "longitude": 23.748804,
                    "bearing": 9.0,
                    "speed": 4.9444447
                },
                "currentStopSequence": 6,
                "currentStatus": "IN_TRANSIT_TO",
                "timestamp": "1732310038",
                "congestionLevel": "UNKNOWN_CONGESTION_LEVEL",
                "stopId": "7551",
                "

# Envoyer vers kafka

In [23]:
import os
import base64
import requests
import json
from google.transit import gtfs_realtime_pb2  # GTFS Realtime
from google.protobuf.json_format import MessageToJson
from confluent_kafka import Producer

# Définir les variables d'environnement
os.environ["CLIENT_ID"] = "6722072041100899"  # Remplacez par votre véritable Client ID
os.environ["CLIENT_SECRET"] = "ISo0oqPMkfCB8wvaPSxrfj20EmnuvKwE"  # Remplacez par votre véritable Client Secret

# Vérification des variables d'environnement
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

if not client_id or not client_secret:
    print("Error: CLIENT_ID and CLIENT_SECRET must be set as environment variables.")
    exit()

# Authentification avec Basic Authentication
auth_string = f"{client_id}:{client_secret}"
encoded_auth = base64.b64encode(auth_string.encode()).decode()

# Préparation de l'URL et des en-têtes
city = "tampere"  # Remplacez par votre ville
endpoint = "vehicleposition"  # Endpoint choisi : tripupdate, vehicleposition, ou servicealert
url = f"https://data.waltti.fi/{city}/api/gtfsrealtime/v1.0/feed/{endpoint}"

headers = {
    "Authorization": f"Basic {encoded_auth}",
}

# Configuration du producteur Kafka
kafka_config = {
    'bootstrap.servers': 'localhost:9092',  # Adresse du broker Kafka
    'client.id': 'python-producer'
}
producer = Producer(kafka_config)
kafka_topic = "gtfs_realtime_topic"  # Remplacez par votre topic Kafka

# Callback pour la livraison Kafka
def delivery_report(err, msg):
    if err is not None:
        print(f"Message delivery failed: {err}")
    else:
        print(f"Message delivered to {msg.topic()} [partition: {msg.partition()}]")

# Faire la requête API
try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Lève une exception pour les erreurs HTTP

    # Parsing des données Protobuf et conversion en JSON
    feed = gtfs_realtime_pb2.FeedMessage()
    feed.ParseFromString(response.content)
    feed_json = json.loads(MessageToJson(feed))

    # Afficher les données localement
    print(json.dumps(feed_json, indent=4))

    # Envoyer les données à Kafka
    for entity in feed.entity:
        vehicle_json = json.loads(MessageToJson(entity))
        producer.produce(
            kafka_topic,
            json.dumps(vehicle_json),
            callback=delivery_report
        )

    # Assurez-vous que tous les messages sont bien envoyés
    producer.flush()

except requests.exceptions.RequestException as e:
    print(json.dumps({"error": f"Error connecting to the API: {e}"}, indent=4))
except Exception as e:
    print(json.dumps({"error": f"Unexpected error: {e}"}, indent=4))


{
    "header": {
        "gtfsRealtimeVersion": "2.0",
        "incrementality": "FULL_DATASET",
        "timestamp": "1732310218"
    },
    "entity": [
        {
            "id": "47374_43",
            "vehicle": {
                "trip": {
                    "tripId": "73_14674_8544710",
                    "startTime": "23:10:00",
                    "startDate": "20241122",
                    "scheduleRelationship": "SCHEDULED",
                    "routeId": "5047374",
                    "directionId": 0
                },
                "position": {
                    "latitude": 61.34435,
                    "longitude": 23.755926,
                    "bearing": 21.0,
                    "speed": 8.388889
                },
                "currentStopSequence": 9,
                "currentStatus": "IN_TRANSIT_TO",
                "timestamp": "1732310218",
                "congestionLevel": "UNKNOWN_CONGESTION_LEVEL",
                "stopId": "7543",
                "